In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn.svm import SVC

In [3]:
df1 = pd.read_csv('{파일경로}/lifelog_data_2018.csv')
df2 = pd.read_csv('{파일경로}/lifelog_data_2019.csv')
df3 = pd.read_csv('{파일경로}/lifelog_data_2020.csv')

In [4]:
def engineering(n, d):
    # 성별 인코딩
    d['gender'] = d['gender'].apply(lambda x:0 if x == 'M' else 1)
    # 날짜 변환
    d['date'] = pd.to_datetime(d['date'])

    # 스트레스(target)
    d['pmStress'] = d['pmStress'] - 1
    # XGBoost의 경우는 0부터 시작 > 나중에 좀 더 찾아보기
    # Stacking을 위해 모든 모델 pmStress 범위 변경
    
    # 감정변화비율 = 오후감정/오전감정
    d['emotionChangeRate'] = d['pmEmotion'] / d['amEmotion']
    # 긍정변화평균 = emotion1~7의 평균
    d['positiveMean'] = d.filter(regex='Positive').mean(axis=1)
    # 긴장변화평균 = emotion1~7의 평균
    d['tensionnMean'] = d.filter(regex='Tension').mean(axis=1)
    
    # 긍정비율 = Positive5~7 / 1~3 > 4는 중앙값으로 진행하지 않음
    d['positiveRate'] = d[['emotionPositive5', 'emotionPositive6', 'emotionPositive7']].sum(axis=1) / d[['emotionPositive1', 'emotionPositive2', 'emotionPositive3']].sum(axis=1)
    # 긴장비율 = Tension5~7 / 1~3 > 4는 중앙값으로 진행하지 않음
    d['tensionRate'] = d[['emotionTension5', 'emotionTension6', 'emotionTension7']].sum(axis=1) / d[['emotionTension1', 'emotionTension2', 'emotionTension3']].sum(axis=1)
    
    # 긍정감정의 임시 테이블
    pos_temp = d.filter(regex='Positive')
    # 가장 높은 긍정감정의 숫자(1~7)
    d['topPositive'] = pos_temp.idxmax(axis=1).apply(lambda x:int(x[-1]))
    # 가장 낮은 긍정감정의 숫자(1~7)
    d['botPositive'] = pos_temp.idxmin(axis=1).apply(lambda x:int(x[-1]))
    
    # 긍정 수치 1~7 * count 수
    for p, c in enumerate(pos_temp.columns):
        d.loc[:, [c]] = d.loc[:, [c]] * (p+1)
        
    del pos_temp
    
    # 긴장감정의 임시 테이블
    ten_temp = d.filter(regex='Tension')
    # 가장 높은 긴장감정의 숫자(1~7)
    d['topTension'] = ten_temp.idxmax(axis=1).apply(lambda x:int(x[-1]))
    # 가장 낮은 긴장감정의 숫자(1~7)
    d['botTension'] = ten_temp.idxmin(axis=1).apply(lambda x:int(x[-1]))
    
    # 긴장 수치 1~7 * count 수
    for t, c in enumerate(ten_temp.columns):
        d.loc[:, [c]] = d.loc[:, [c]] * (t+1)
    
    del ten_temp
    
    # 수치 반영 긍정 평균
    d['positiveWMean'] = d.filter(regex='Positive').mean(axis=1)
    # 수치 반영 긴장 평균
    d['tensionWMean'] = d.filter(regex='Tension').mean(axis=1)
    
    # 수치 반영 긍정 비율
    d['positiveWRate'] = d[['emotionPositive5', 'emotionPositive6', 'emotionPositive7']].sum(axis=1) / d[['emotionPositive1', 'emotionPositive2', 'emotionPositive3']].sum(axis=1)
    # 수치 반영 긴장 비율
    d['tensionWRate'] = d[['emotionTension5', 'emotionTension6', 'emotionTension7']].sum(axis=1) / d[['emotionTension1', 'emotionTension2', 'emotionTension3']].sum(axis=1)
    
    # 최고 수치를 반영한 오전 감정 = (오전 감정 * (최고 감정 + 최고 긴장)) / 14
    d['aCtPT'] = round(d['amEmotion'] * (d['topPositive'] + d['topTension']) / 14)
    # 최고 수치를 반영한 오전 컨디션 = (오전 감정 * (최고 감정 + 최고 긴장)) / 14
    d['aEtPT'] = round(d['amCondition'] * (d['topPositive'] + d['topTension']) / 14)
    # 최고 수치를 반영한 오후 감정 = (오전 감정 * (최고 감정 + 최고 긴장)) / 14
    d['pEtPT'] = round(d['pmEmotion'] * (d['topPositive'] + d['topTension']) / 14)
    
    d['aCEpEtPTm'] = (d['aCtPT'] + d['aEtPT'] + d['pEtPT'])
    
    # 긍정적인지 = 수치 반영 긍정 평균이 중앙값보다 크면 1 아니면 0
    d['positive'] = d['positiveWRate'].apply(lambda x:1 if x > d['positiveWRate'].median() else 0)
    # 부정적인지 = 수치 반영 긍정 평균이 중앙값보다 작으면 1 아니면 0
    d['negative'] = d['positiveWRate'].apply(lambda x:1 if x < d['positiveWRate'].median() else 0)
    
    # 긴장 상태인지 = 수치 반영 긴장 평균이 중앙값보다 크면 1 아니면 0
    d['aroused'] = d['tensionWRate'].apply(lambda x:1 if x > d['tensionWRate'].median() else 0)
    # 편안한 상태인지 = 수치 반영 긴장 평균이 중앙값보다 작으면 1 아니면 0
    d['relaxed'] = d['tensionWRate'].apply(lambda x:1 if x < d['tensionWRate'].median() else 0)
    
    # 활동 비율 = 자전거, 도보의 합 / 운송수단, 가만히 있기의 합
    d['activityRate'] = d[['on_bicycle', 'on_foot']].sum(axis=1) / d[['in_vehicle', 'still']].sum(axis=1)
    # 0으로 나누기된 것을 0으로 치환 > inf 값을 가짐
    d.replace([np.inf, -np.inf], 0, inplace=True)
    d.dropna(inplace=True)
    d.sample(n=d.shape[0], random_state=47)
    
    return d

In [5]:
for n, d in enumerate([df1, df2, df3]):
    globals() [f'df{n+1}'] = engineering(n, d.copy()) # 위에서 생성한 engineering 함수 적용
df = pd.concat([df1, df2, df3], axis=0) # 데이터 결합 
df = df[[c for c in df.columns if c not in ['pmStress']] + ['pmStress']] # pmStress 마지막 열로 이동
df.drop(['date', 'userId'], axis = 1, inplace = True) # dtae, userId 삭제

In [6]:
# 하이퍼파라미터
random_state = 1234
c_value = [100]
gamma_value = [0.01]
kernels = ['sigmoid']
dfs = ['ovo']
test_accuracy_list = []
results = []

# smote = SMOTE()

X, y = df.iloc[:, :-1], df.iloc[:, -1]

for i in range(30):
    train = pd.DataFrame()
    valid = pd.DataFrame()
    test = pd.DataFrame()
    for j in range(5):
        train = pd.concat([train, df.loc[df['pmStress'] == np.unique(df['pmStress'])[j]][:round(len(df) / 5 * 0.9)]], axis=0) # 90% test data
        test = pd.concat([test, df.loc[df['pmStress'] == np.unique(df['pmStress'])[j]][-round(len(df) / 5 * 0.1):]], axis=0) # 10% test data

    train = train.sample(n=train.shape[0]) # , random_state=1234)
    test = test.sample(n=test.shape[0]) # , random_state=1234)
    
    # X_train, y_train = smote.fit_resample(train.iloc[:, :-1], train.iloc[:, -1])
    X_train, y_train = train.iloc[:, :-1], train.iloc[:, -1]
    X_test, y_test = test.iloc[:, :-1], test.iloc[:, -1]
    
    best_score = 0
    best_params = None
    for C in c_value:
        for gamma in gamma_value:
            for kernel in kernels:
                for decision_function_shape in dfs:
                    score_sum = 0
                    # 데이터를 5개의 fold로 나누고, Shuffle = True로 지정하여 데이터를 랜덤으로 섞음
                    # k-fold 교차검증을 수행하기 위해 train set과 validation data로 분리
                    for train_index, val_index in KFold(n_splits=5, shuffle=True, random_state=random_state).split(train):
                        X_train_kf, X_val_kf = X_train.iloc[train_index], X_train.iloc[val_index]
                        y_train_kf, y_val_kf = y_train.iloc[train_index], y_train.iloc[val_index]
                        clf = SVC(C=C, gamma=gamma, kernel=kernel, decision_function_shape=decision_function_shape)
                        clf.fit(X_train_kf, y_train_kf)
                        score_sum += accuracy_score(y_val_kf, clf.predict(X_val_kf))
                    score_avg = score_sum / 5  # 5번 교차검증 정확도의 평균
                    if score_avg > best_score:# best_score보다 score_sum이 클 경우 best_score를 score_avg로 업데이트하고 헤딩 파라미터 조합을 best_params에 저장
                        best_score = score_avg # best_score와 score_avg 값이 동일할 경우, False
                        # 각 하이퍼파라미터의 값을 출력

                        best_params = {'C': C, 'gamma': gamma, 'kernel': kernel, 'decision_function_shape': decision_function_shape}
                    print(f"C: {C}, gamma: {gamma}, kernel: {kernel}, decision_function_shape: {decision_function_shape}",
                          f" -> accuracy: {score_avg}")
    # 하이퍼 파라미터 값을 갖고 SVC 모델 적합      
    clf = SVC(**best_params)
    clf.fit(X_train_kf, y_train_kf)
    # 하이퍼 파라미터 값을 갖고, test data로 예측한 y_pred 값
    y_pred_SVC = clf.predict(X_test)
    # 실제 test값과 예측값 사이의 정확도
    accuracy = accuracy_score(y_test, y_pred_SVC)
    results.append(accuracy)

    print(f"Iteration: {i+1}") # 반복 횟수
    print(f"Best params: {best_params}") # 하이퍼파라미터 조합
    print(f"Accuracy on test set: {accuracy}") # accuracy
    print('=='*70)

    test_accuracy_list.append(accuracy) # 반복횟수에 따른 정확도를 저장

# result_SVC = pd.DataFrame({'Test Accuracy': test_accuracy_list}) # 데이터프레임 형식으로 test_accuracy_list 저장                           

C: 100, gamma: 0.01, kernel: sigmoid, decision_function_shape: ovo  -> accuracy: 0.2126582278481013
Iteration: 1
Best params: {'C': 100, 'gamma': 0.01, 'kernel': 'sigmoid', 'decision_function_shape': 'ovo'}
Accuracy on test set: 0.2
C: 100, gamma: 0.01, kernel: sigmoid, decision_function_shape: ovo  -> accuracy: 0.21139240506329116
Iteration: 2
Best params: {'C': 100, 'gamma': 0.01, 'kernel': 'sigmoid', 'decision_function_shape': 'ovo'}
Accuracy on test set: 0.2
C: 100, gamma: 0.01, kernel: sigmoid, decision_function_shape: ovo  -> accuracy: 0.1911392405063291
Iteration: 3
Best params: {'C': 100, 'gamma': 0.01, 'kernel': 'sigmoid', 'decision_function_shape': 'ovo'}
Accuracy on test set: 0.2
C: 100, gamma: 0.01, kernel: sigmoid, decision_function_shape: ovo  -> accuracy: 0.19367088607594937
Iteration: 4
Best params: {'C': 100, 'gamma': 0.01, 'kernel': 'sigmoid', 'decision_function_shape': 'ovo'}
Accuracy on test set: 0.2
C: 100, gamma: 0.01, kernel: sigmoid, decision_function_shape: ovo

In [161]:
y_pred_SVC = pd.DataFrame(y_pred_SVC)
y_pred_SVC.columns = ['SVC']
y_pred_SVC.to_csv('SVC.csv')

In [12]:
from sklearn.model_selection import train_test_split

# 하이퍼파라미터
random_state = 1234
c_value = [100]
gamma_value = [0.01]
kernels = ['sigmoid']
dfs = ['ovo']
test_accuracy_list = []
results = []

for i in range(20):
    X, y = df.iloc[:, :-1], df.iloc[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = random_state)

    best_score = 0
    best_params = None
    for C in c_value:
        for gamma in gamma_value:
            for kernel in kernels:
                for decision_function_shape in dfs:
                    score_sum = 0
                    # 데이터를 5개의 fold로 나누고, Shuffle = True로 지정하여 데이터를 랜덤으로 섞음
                    # k-fold 교차검증을 수행하기 위해 train set과 validation data로 분리
                    for train_index, val_index in KFold(n_splits=5, shuffle=True, random_state=random_state).split(train):
                        X_train_kf, X_val_kf = X_train.iloc[train_index], X_train.iloc[val_index]
                        y_train_kf, y_val_kf = y_train.iloc[train_index], y_train.iloc[val_index]
                        clf = SVC(C=C, gamma=gamma, kernel=kernel, decision_function_shape=decision_function_shape)
                        clf.fit(X_train_kf, y_train_kf)
                        score_sum += accuracy_score(y_val_kf, clf.predict(X_val_kf))
                    score_avg = score_sum / 5  # 5번 교차검증 정확도의 평균
                    if score_avg > best_score:# best_score보다 score_sum이 클 경우 best_score를 score_avg로 업데이트하고 헤딩 파라미터 조합을 best_params에 저장
                        best_score = score_avg # best_score와 score_avg 값이 동일할 경우, False
                        # 각 하이퍼파라미터의 값을 출력

                        best_params = {'C': C, 'gamma': gamma, 'kernel': kernel, 'decision_function_shape': decision_function_shape}
                    print(f"C: {C}, gamma: {gamma}, kernel: {kernel}, decision_function_shape: {decision_function_shape}",
                          f" -> accuracy: {score_avg}")
    # 하이퍼 파라미터 값을 갖고 SVC 모델 적합      
    clf = SVC(**best_params)
    clf.fit(X_train_kf, y_train_kf)
    # 하이퍼 파라미터 값을 갖고, test data로 예측한 y_pred 값
    y_pred_SVC = clf.predict(X_test)
    # 실제 test값과 예측값 사이의 정확도
    accuracy = accuracy_score(y_test, y_pred_SVC)
    results.append(accuracy)

    print(f"Iteration: {i+1}") # 반복 횟수
    print(f"Best params: {best_params}") # 하이퍼파라미터 조합
    print(f"Accuracy on test set: {accuracy}") # accuracy
    print('=='*70)

    test_accuracy_list.append(accuracy) # 반복횟수에 따른 정확도를 저장

# result_SVC = pd.DataFrame({'Test Accuracy': test_accuracy_list}) # 데이터프레임 형식으로 test_accuracy_list 저장                           

C: 100, gamma: 0.01, kernel: sigmoid, decision_function_shape: ovo  -> accuracy: 0.2645569620253164
Iteration: 1
Best params: {'C': 100, 'gamma': 0.01, 'kernel': 'sigmoid', 'decision_function_shape': 'ovo'}
Accuracy on test set: 0.2815533980582524
C: 100, gamma: 0.01, kernel: sigmoid, decision_function_shape: ovo  -> accuracy: 0.2987341772151899
Iteration: 2
Best params: {'C': 100, 'gamma': 0.01, 'kernel': 'sigmoid', 'decision_function_shape': 'ovo'}
Accuracy on test set: 0.32038834951456313
C: 100, gamma: 0.01, kernel: sigmoid, decision_function_shape: ovo  -> accuracy: 0.25569620253164554
Iteration: 3
Best params: {'C': 100, 'gamma': 0.01, 'kernel': 'sigmoid', 'decision_function_shape': 'ovo'}
Accuracy on test set: 0.24271844660194175
C: 100, gamma: 0.01, kernel: sigmoid, decision_function_shape: ovo  -> accuracy: 0.29113924050632906
Iteration: 4
Best params: {'C': 100, 'gamma': 0.01, 'kernel': 'sigmoid', 'decision_function_shape': 'ovo'}
Accuracy on test set: 0.27184466019417475
C: 